In [5]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo
import os
from config import nasa_url, featured_img_url, facts_url, mars_hemisphere_url

In [6]:
# URL of page to be scraped
url = nasa_url

# Retrieve page with the requests module
response = requests.get(url, verify=True)